# 数据源

对于机器学习来说，数据是至关重要的，因此，第一步我们将讨论数据来源。  
本章将主要讨论3个数据来源：文件、数据库和API。  

## 文件

文件是
常见的数据来源之一，例如Excel文件、csv文件等。  
`pandas`库提供了读取多种常用文件类型的方法，例如

- `pandas.read_csv`：读取csv文件，可以通过设置`sep`可以读取不同分隔符的文件，例如`tab`为分割符的文件
- `pandas.read_excel`：读取Excel文件（需要安装`openpyxl`库）
- `pandas.read_json`：读取JSON文件
- `pandas.read_sas`：读取SAS文件
- `pandas.read_ssps`：读取SPSS文件

In [1]:
# 利用pandas.read_csv方法来读取一个csv文件

import pandas as pd

df = pd.read_csv('data/nba_players.csv')
df.head()

,player_id,player_name,avg_pts,avg_ast,avg_oreb,avg_dreb,avg_stl,avg_blk,avg_tov,avg_fgm,avg_fga,avg_tpm,avg_play_time,team_name,position
0,1,凯尔-科沃尔,10.12,1.64,0.13,2.66,0.49,0.31,1.03,3.57,7.66,2.42,26.13,骑士,2.5
1,2,蒂亚戈-斯普利特,4.33,0.44,0.89,1.56,0.11,0.11,0.67,1.56,3.44,0.22,8.56,76人,5.0
2,3,保罗-米尔萨普,17.87,3.63,1.59,6.13,1.30,0.87,2.29,6.17,13.94,1.09,33.66,老鹰,4.0
3,4,萨博-塞福洛沙,7.16,1.73,0.84,3.52,1.48,0.50,0.95,2.81,6.35,0.66,25.73,老鹰,2.5
4,5,杰夫-蒂格,15.29,7.78,0.39,3.66,1.24,0.39,2.63,4.90,11.07,1.10,32.44,步行者,1.0


此外，`pandas`还提供了一个`read_clipboard`方法，可以方便读取你`复制`中的内容，例如我们复制一下内容:  

>
date,h,l,w  
2020-01-01,19,10,rainy  
2020-01-02,18,11,sunny  
2020-01-03,17,10,sunny  
>

In [3]:
# 读取复制中的内容

pd.read_clipboard(sep=',')

,date,h,l,w
0,2020-01-01,19,10,rainy
1,2020-01-02,18,11,sunny
2,2020-01-03,17,10,sunny


这样，我们可以方便的复制Excel中的数据，然后使用`read_clipboard`方法，可以快速读取数据  
例如我们复制下图中Excel的数据

![excel_screenshot](resource/chp1_excel_clipboard.png)

In [6]:
# 读取复制的Excel表中的内容

pd.read_clipboard(sep='\t')

,id,name,score
0,1,Marry,8.2
1,2,Peter,9.1
2,3,Jhon,8.6


## 数据库

数据库则是数据最主要的来源，特别在工程环境下，大部分数据都来源于数据库。`pandas`库中提供了`read_sql`方法，可以通过sql语句来读取数据库中的数据，这是我们最常用的读取数据库中数据并转换成`DataFrame`的方法。  
`read_sql`需要设置两个必要参数，`sql`和`con`

- `sql`：sql语句  
- `con`：数据库的链接  
    创建获取数据库链接，我们可以使用数据库相应的库来完成，例如`sqlite3`、`pymysql`等  

例如，我们从本地的`MySQL`数据库读取`user`表中的数据

~~~Python
import pymysql
import pandas as pd

# 创建MySQL链接
conn = pymysql.connect(
    host='127.0.0.1',   # MySQL地址
    port=3306,          # MySQL端口
    user='root',        # MySQL用户名
    password='123456',  # MySQL密码
    database='test'     # MySQL数据库
)
sql = 'select * from user'
data = pd.read_sql(sql, conn)
# 关闭MySQL链接
conn.close() 
~~~

此外还有一个比较常用的参数是`parse_dates`，设置这个参数可以将指定的列转化为日期类型。

In [10]:
# 我们读取SQLite库中的user表中的全部数据

import sqlite3

# SQLite数据在data文件夹下
conn = sqlite3.connect('data/sqlite_db.db')
df = pd.read_sql('SELECT * FROM user', conn)
conn.close()

df.head()

,id,name,score,level
0,1,Mary,9.3,A
1,2,Peter,8.1,B
2,3,Jhon,8.2,B
3,4,Jane,7.4,C


## API

调用API获取数据也是数据重要的来源之一，例如事实的股票数据，天气数据等，可以通过调用提供这些查询服务的API来获取。  
很多公司内部系统也会通过API形式来提供数据，这样做可以使得内部系统耦合度降低。假设A公司的平台系统中有许多子系统组成，其中用户平台系统可以提供用户信息API，其他子系统可以调用该API获取数据，当用户平台内部的需要做一些修改时，例如对数据库的增减进行字段，只要保持API请求参数和响应参数不变，则不会影响其他子系统调用用户信息数据。  
`requests`库提供了一些常用的请求接口的方法，现在大部分API返回的都是`JSON`数据格式，因此可以使用内置的`json`库来方便的处理这些返回数据。例如下面是一个通过使用`requests`库从天气查询API中获取天气数据，然后使用`json`库对返回数据进行处理，最后转为一个`DataFrame`

In [3]:
# 从API获取数据

import json
import requests

# 调用API获取上海的天气数据
response = requests.get('http://wthrcdn.etouch.cn/weather_mini?city=上海')
# 使用json.loads函数将json格式数据转为dict
content = json.loads(response.content)
content

{'data': {'yesterday': {'date': '8日星期三',
   'high': '高温 7℃',
   'fx': '西北风',
   'low': '低温 4℃',
   'fl': '<![CDATA[3-4级]]>',
   'type': '多云'},
  'city': '上海',
  'aqi': '48',
  'forecast': [{'date': '9日星期四',
    'high': '高温 7℃',
    'fengli': '<![CDATA[<3级]]>',
    'low': '低温 6℃',
    'fengxiang': '北风',
    'type': '小雨'},
   {'date': '10日星期五',
    'high': '高温 9℃',
    'fengli': '<![CDATA[3-4级]]>',
    'low': '低温 7℃',
    'fengxiang': '东北风',
    'type': '小雨'},
   {'date': '11日星期六',
    'high': '高温 9℃',
    'fengli': '<![CDATA[3-4级]]>',
    'low': '低温 5℃',
    'fengxiang': '东北风',
    'type': '小雨'},
   {'date': '12日星期天',
    'high': '高温 9℃',
    'fengli': '<![CDATA[3-4级]]>',
    'low': '低温 4℃',
    'fengxiang': '北风',
    'type': '阴'},
   {'date': '13日星期一',
    'high': '高温 10℃',
    'fengli': '<![CDATA[3-4级]]>',
    'low': '低温 5℃',
    'fengxiang': '东风',
    'type': '阴'}],
  'ganmao': '天冷空气湿度大，易发生感冒，请注意适当增加衣服，加强自我防护避免感冒。',
  'wendu': '7'},
 'status': 1000,
 'desc': 'OK'}

In [4]:
# 获取天气预报的信息

forecast = content['data']['forecast']
df = pd.DataFrame(forecast)
df

,date,high,fengli,low,fengxiang,type
0,9日星期四,高温 7℃,<![CDATA[<3级]]>,低温 6℃,北风,小雨
1,10日星期五,高温 9℃,<![CDATA[3-4级]]>,低温 7℃,东北风,小雨
2,11日星期六,高温 9℃,<![CDATA[3-4级]]>,低温 5℃,东北风,小雨
3,12日星期天,高温 9℃,<![CDATA[3-4级]]>,低温 4℃,北风,阴
4,13日星期一,高温 10℃,<![CDATA[3-4级]]>,低温 5℃,东风,阴


我们也可以同时获取多个城市的预报信息

In [16]:
# 获取多个城市的天气预报信息

city_list = ['上海', '北京']
df_list = []

for c in city_list:
    url = f'http://wthrcdn.etouch.cn/weather_mini?city={c}'
    response = requests.get(url)
    content = json.loads(response.content)
    df = pd.DataFrame(content['data']['forecast'])
    df['city'] = c
    df_list.append(df)

pd.concat(df_list)

,date,high,fengli,low,fengxiang,type,city
0,9日星期四,高温 7℃,<![CDATA[<3级]]>,低温 6℃,北风,小雨,上海
1,10日星期五,高温 9℃,<![CDATA[3-4级]]>,低温 7℃,东北风,小雨,上海
2,11日星期六,高温 9℃,<![CDATA[3-4级]]>,低温 5℃,东北风,小雨,上海
3,12日星期天,高温 9℃,<![CDATA[3-4级]]>,低温 4℃,北风,阴,上海
4,13日星期一,高温 10℃,<![CDATA[3-4级]]>,低温 5℃,东风,阴,上海
0,9日星期四,高温 5℃,<![CDATA[<3级]]>,低温 -7℃,西南风,晴,北京
1,10日星期五,高温 6℃,<![CDATA[<3级]]>,低温 -6℃,无持续风向,晴,北京
2,11日星期六,高温 2℃,<![CDATA[<3级]]>,低温 -6℃,东北风,多云,北京
3,12日星期天,高温 3℃,<![CDATA[<3级]]>,低温 -6℃,北风,多云,北京
4,13日星期一,高温 2℃,<![CDATA[<3级]]>,低温 -6℃,北风,阴,北京


## Landing

本章`Landing`小节将会带来2个实战部分：

- API：我们将封装天气数据API，每次我们输入一个城市列表，输出这些城市的天气预报
- 数据库：数据库部分我们将封装一个`SQLite`的类，方便我们操作`SQLite`数据库

### API

将一些常用的操作写在一个方法里面，是一种常用的封装操作，这样可以方便的复用经常使用的代码块。  
本例中，我们将调用天气数据API获取天气预报数据的代码块封装成一个方法，该方法的目标是输入一个城市列表，如`['上海', '北京', '广州']`，输出该列表上城市的天气预报数据。

我们将该封装的方法写在一个python文件中作为一个单独python的模块。在这个模块中，我们将按照以下三个部分来组织我们的代码。  

1. 引入所需要的库。一般模块中所用到的库，我们都在写文件开头，并且分为3个部分：
    - 系统自带的库
    - 第三方库
    - 自己编写的库/模块等
   

2. 一些可配置的常量。例如API的URL，数据库的用户名、数据库地址等，可以写在这部分。配置这些常量，将会有几个好处：
    - 如果文件中有多个地方需要使用到该配置，那么我们只需要修改这个变量就可以了，不容易出现修改错误、遗漏修改等问题
    - 当我们需要修改某些常量时，在文件头部位置就能找到这些两边并且修改，非常方便


3. 功能实现代码。

第一部分我们引入需要使用到的库

In [19]:
# 引入需要的库

import json

import pandas as pd
import requests

第二部分我们配置常量，这里将会配置一个API地址的常量

In [20]:
# 配置API地址

URL = 'http://wthrcdn.etouch.cn/weather_mini'

第三部分则是我们的功能实现的代码。在本例中，我们可以将查询单个城市的天气预报数据的功能封装成一个函数，然后可以基于这个方法来实现多个城市的查询。

In [34]:
def get_city_forecast(city):
    """
    获取城市的天气预报
    :param city: 城市
    :return: 该城市的天气预报
    """
    response = requests.get(URL, params={'city': city})
    content = json.loads(response.content)
    df = pd.DataFrame(content['data']['forecast'])
    df['city'] = city
    
    return df

In [33]:
get_city_forecast('上海')

,date,high,fengli,low,fengxiang,type,city
0,10日星期五,高温 9℃,<![CDATA[3-4级]]>,低温 7℃,东北风,小雨,上海
1,11日星期六,高温 9℃,<![CDATA[3-4级]]>,低温 5℃,东北风,小雨,上海
2,12日星期天,高温 9℃,<![CDATA[3-4级]]>,低温 4℃,无持续风向,阴,上海
3,13日星期一,高温 10℃,<![CDATA[3-4级]]>,低温 5℃,东风,阴,上海
4,14日星期二,高温 10℃,<![CDATA[<3级]]>,低温 6℃,东北风,小雨,上海
